In [9]:
import torch
import torch.nn as nn
import math

In [10]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        """
        위치 인코딩 클래스를 초기화합니다. 이 클래스는 트랜스포머 모델에 위치 정보를 제공합니다.

        :param d_model: 모델의 임베딩 차원입니다. 각 단어 또는 토큰이 이 차원으로 표현됩니다.
        :param dropout: 드롭아웃 비율입니다. 학습 중에 위치 인코딩을 적용한 후 일부 정보를 무작위로 0으로 설정합니다.
        :param max_len: 시퀀스에서 고려할 최대 길이입니다. 이 값은 모든 가능한 위치에 대한 인코딩을 미리 계산하는 데 사용됩니다.
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 각 위치에 대한 인코딩을 계산합니다.
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        
        # 짝수 인덱스에는 사인 함수를, 홀수 인덱스에는 코사인 함수를 적용합니다.
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # 위치 인코딩 텐서의 크기를 조정합니다. (시퀀스 길이, 1, 임베딩 차원)
        pe = pe.unsqueeze(0).transpose(0, 1)
        
        # 위치 인코딩을 모델의 버퍼로 등록합니다. 이는 학습 과정에서 업데이트되지 않지만, 모델 저장 및 로드 시에 함께 저장되고 로드됩니다.
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        모델의 순방향 전파 중에 호출됩니다. 입력 텐서에 위치 인코딩을 더한 후 드롭아웃을 적용합니다.

        :param x: 입력 텐서. 크기는 (시퀀스 길이, 배치 크기, 임베딩 차원)입니다.
        :return: 위치 인코딩이 적용된 후 드롭아웃이 적용된 텐서를 반환합니다.
        """
        # 입력 텐서에 위치 인코딩을 더합니다.
        x = x + self.pe[:x.size(0), :]
        
        # 드롭아웃을 적용합니다.
        return self.dropout(x)

In [11]:
class TransformerModel(nn.Module):
    def __init__(self, ntoken, d_model, nhead, nhid, nlayers, dropout=0.5):
        """
        Transformer 모델의 생성자입니다. 이 클래스는 Transformer 모델을 초기화합니다.

        :param ntoken: 입력 및 출력 어휘 사전의 크기입니다.
        :param d_model: 트랜스포머 내부에서 사용되는 피처(임베딩) 벡터의 차원입니다.
        :param nhead: 멀티헤드 어텐션에서의 헤드(head) 수입니다.
        :param nhid: 피드포워드 신경망의 차원입니다.
        :param nlayers: 트랜스포머 인코더 층의 수입니다.
        :param dropout: 드롭아웃 비율입니다.
        """
        super(TransformerModel, self).__init__()
        self.d_model = d_model  # 임베딩 차원 저장
        
        # 포지셔널 인코딩 레이어를 초기화합니다.
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        # Transformer 레이어를 초기화합니다.
        self.transformer = nn.Transformer(d_model, nhead, nlayers, nlayers, nhid, dropout)
        
        # 임베딩 레이어를 초기화합니다.
        self.encoder = nn.Embedding(ntoken, d_model)
        
        # Transformer 출력을 어휘 사전 크기로 매핑하는 디코더 레이어를 초기화합니다.
        self.decoder = nn.Linear(d_model, ntoken)
        
        # 모델 가중치를 초기화하는 메서드를 호출합니다.
        self.init_weights()
    
    def init_weights(self):
        """
        Transformer 모델의 가중치를 초기화합니다.
        """
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def generate_square_subsequent_mask(self, sz):
        """
        디코더 입력에 사용될 후속 위치 마스킹을 생성합니다.

        :param sz: 시퀀스 길이
        :return: 마스크 텐서
        """
        mask = torch.triu(torch.ones(sz, sz), diagonal=1).type_as(self.encoder.weight.data)
        mask = mask.masked_fill(mask == 1, float('-inf'))
        return mask
    
    def forward(self, src, src_mask=None):
        """
        모델의 순방향 전파를 수행합니다.

        :param src: 입력 시퀀스
        :param src_mask: 선택적으로, 소스 시퀀스에 적용될 마스크
        :return: 모델의 출력
        """
        if src_mask is None:
            # 마스크가 제공되지 않은 경우 생성합니다.
            src_mask = self.generate_square_subsequent_mask(src.size(0))
        
        # 입력 시퀀스에 임베딩과 포지셔널 인코딩을 적용합니다.
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        
        # Transformer를 통해 입력 데이터 처리
        output = self.transformer(src, src, src_mask=src_mask)
        
        # 최종 출력을 생성하기 위해 디코더를 적용
        output = self.decoder(output)
        return output

In [12]:
# 예제 입력
ntokens = 1000  # 어휘 사전의 크기
d_model = 512  # 임베딩 차원
nhead = 8  # 멀티헤드 어텐션의 헤드 수
nhid = 2048  # 피드포워드 네트워크의 차원
nlayers = 6  # 인코더 레이어 수

In [13]:
dropout = 0.5  # 드롭아웃 비율

# 모델 인스턴스 생성
model = TransformerModel(ntokens, d_model, nhead, nhid, nlayers, dropout)

# 입력 데이터 준비
# 여기서는 임의의 정수로 구성된 입력 시퀀스를 생성합니다. 실제 어플리케이션에서는 토큰화된 텍스트 데이터를 사용하게 됩니다.
src = torch.randint(0, ntokens, (10, 32))  # (시퀀스 길이, 배치 크기)
src_mask = model.generate_square_subsequent_mask(src.size(0)).to(src.device)

# 모델 실행
output = model(src, src_mask)

print(output.size())  # 예상 출력 크기: (시퀀스 길이, 배치 크기, 어휘 사전 크기)


# Mask 생성 및 적용 예시
sz = 10  # 시퀀스 길이
mask = generate_square_subsequent_mask(sz)
print(mask)


torch.Size([10, 32, 1000])
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [14]:
#결과의미: 트랜스포머 모델 내에서 사용되는 "마스킹" 기법을 나타냅니다.
#0. (또는 0)은 해당 위치의 정보가 "보인다(접근 가능하다)"는 것을 의미
#-inf는 해당 위치의 정보가 "보이지 않는다(접근 불가능하다)"는 것을 의미
#예를 들어, 3번째 행을 보면, 첫 세 개의 위치는 0으로 표시되어 있어 이들 위치의 정보를 "볼 수 있음"을 의미하고, 
#나머지 위치는 -inf로 표시되어 있어 "볼 수 없음"을 나타냅니다. 
#이는 디코더가 3번째 단어를 예측할 때, 첫 두 단어의 정보만 사용할 수 있음을 의미합니다
#이런 방식으로 마스킹을 적용함으로써, 디코더는 각 단계에서 올바른 정보만을 사용하여 다음 단어를 예측할 수 있게 됩니다